<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [4]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [5]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [6]:
## Task 1: Mark all launch sites on a map
# Create a map centered at a specific coordinate
latitude = 28.573255
longitude = -80.646895
launch_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# Display the map
launch_map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [7]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [12]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [11]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [10]:
# Create a map centered at NASA Johnson Space Center, Houston, Texas
latitude = 29.559684
longitude = -95.083097
launch_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# Add launch site markers to the map
for index, row in launch_sites_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Launch Site'],
        icon=folium.Icon(color='blue', icon='rocket')
    ).add_to(launch_map)

# Display the map
launch_map


<class 'NameError'>: name 'launch_sites_df' is not defined

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [13]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [14]:
# Coordinates for NASA Johnson Space Center
nasa_coordinate = [29.559684, -95.083097]

# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(
    location=nasa_coordinate, 
    radius=1000, 
    color='#d35400', 
    fill=True
).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a blue circle at NASA Johnson Space Center's coordinate with an icon showing its name
marker = folium.map.Marker(
    location=nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color: #d35400;"><b>NASA JSC</b></div>'
    )
)

# Add the circle and marker to the map
launch_map.add_child(circle)
launch_map.add_child(marker)

# Display the map
launch_map


and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [15]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, row in launch_sites_df.iterrows():
    # Create a Circle object for each launch site
    circle = folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,
        color='#000000',
        fill=True
    ).add_child(folium.Popup(row['Launch Site']))
    
    # Create a Marker object for each launch site
    marker = folium.map.Marker(
        location=[row['Lat'], row['Long']],
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % row['Launch Site'],
        )
    )
    
    # Add the Circle and Marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


## EXPLANATIONS

### Are all launch sites in proximity to the Equator line?

By examining the map, you can determine the latitude of each launch site. The Equator line is at latitude 0°, so launch sites near the Equator will have latitudes close to 0°. Sites like Cape Canaveral and Kennedy Space Center are in Florida, which is not extremely close to the Equator but still relatively closer compared to higher latitudes.

### Are all launch sites in very close proximity to the coast?

Most launch sites are strategically located near the coast. For instance, Cape Canaveral and Kennedy Space Center are very close to the Atlantic Ocean. Coastal locations are often preferred for launch sites to reduce the risk of debris falling on populated areas and to take advantage of the rotational velocity of the Earth, which is higher at the equator and assists with launch efficiency.

### Findings Explanation:

Launch sites tend to be positioned near the coast to ensure safety and maximize the efficiency of launches. Coastal locations provide a safe trajectory over the ocean, minimizing the risk to human life and property from falling debris.

Sites closer to the equator are preferred because the Earth's rotational speed is highest at the equator, providing an additional velocity boost for rockets launched eastward. This makes launches more fuel-efficient, especially for missions aiming to reach geostationary orbits.

In [18]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Define a color function based on the 'class' column, where 1 indicates success and 0 indicates failure
def get_marker_color(launch_status):
    if launch_status == 1:
        return 'green'
    else:
        return 'red'

# For each launch site, add a Circle and Marker object based on its coordinate (Lat, Long) values
for index, row in spacex_df.iterrows():
    # Create a Circle object for each launch site
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,
        color='#000000',
        fill=True
    ).add_child(folium.Popup(row['Launch Site'])).add_to(site_map)
    
    # Create a Marker object for each launch status
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Launch Site'],
        icon=folium.Icon(color=get_marker_color(row['class']), icon='rocket')
    ).add_to(site_map)

# Display the map
site_map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [19]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [20]:
# Import the MarkerCluster plugin
from folium.plugins import MarkerCluster

# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Define a color function based on the 'class' column, where 1 indicates success and 0 indicates failure
def get_marker_color(launch_status):
    if launch_status == 1:
        return 'green'
    else:
        return 'red'

# For each launch record, add a Marker object to the MarkerCluster
for index, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"Launch Site: {row['Launch Site']}\nOutcome: {'Success' if row['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=get_marker_color(row['class']), icon='rocket')
    ).add_to(marker_cluster)

# Display the map
site_map



*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [21]:
# Define a function to assign marker colors based on the class value
def assign_marker_color(launch_status):
    if launch_status == 1:
        return 'green'
    else:
        return 'red'

# Apply the function to create the marker_color column
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

# Display the updated DataFrame
spacex_df.tail(10)


,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [22]:
# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# For each launch record, add a Marker object to the MarkerCluster
for index, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"Launch Site: {row['Launch Site']}\nOutcome: {'Success' if row['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=row['marker_color'], icon='rocket')
    ).add_to(marker_cluster)

# Display the map
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [23]:
# TASK 3: Calculate the distances between a launch site to its proximities
await piplite.install(['geopy'])
from geopy.distance import geodesic

# Define proximities (latitude and longitude) as a dictionary
proximities = {
    'Houston': [29.7604, -95.3698],
    'Dallas': [32.7767, -96.7970],
    'Orlando': [28.5383, -81.3792]
}

# Calculate the distances
distances = {}
for site, row in launch_sites_df.iterrows():
    site_coords = (row['Lat'], row['Long'])
    for proximity, coords in proximities.items():
        distance = geodesic(site_coords, coords).km
        if site not in distances:
            distances[site] = []
        distances[site].append((proximity, distance))

# Display the distances
distances


{0: [('Houston', 1444.229116216735),
  ('Dallas', 1621.0728577037728),
  ('Orlando', 78.51145213095975)],
 1: [('Houston', 1444.2656976779251),
  ('Dallas', 1621.086735123508),
  ('Orlando', 78.56700743958626)],
 2: [('Houston', 1437.3196072538476),
  ('Dallas', 1614.2811536431918),
  ('Orlando', 71.76243701996857)],
 3: [('Houston', 2433.6123691141624),
  ('Dallas', 2212.0355777780837),
  ('Orlando', 3760.4824967155278)]}

Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [24]:
# Import the MousePosition plugin
from folium.plugins import MousePosition

# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add Mouse Position to the map
MousePosition(
    position='topright',
    separator=' | ',
    empty_string='Unavailable',
    lng_first=True,
    num_digits=5,
    prefix='Coordinates:'
).add_to(site_map)

# Display the map
site_map


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [28]:
# Import necessary libraries
from folium.plugins import MousePosition
from math import sin, cos, sqrt, atan2, radians

# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add Mouse Position to the map
MousePosition(
    position='topright',
    separator=' | ',
    empty_string='Unavailable',
    lng_first=True,
    num_digits=5,
    prefix='Coordinates:'
).add_to(site_map)

# Display the map with MousePosition enabled
site_map


In [29]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance


*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [30]:
# Example coordinates for a launch site (e.g., Cape Canaveral)
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# Coordinates for the closest coastline point
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate the distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"The distance between the launch site and the closest coastline is {distance_coastline} km.")


The distance between the launch site and the closest coastline is 7.42932194658424 km.


In [31]:
from folium.features import DivIcon

# Example coordinates for a launch site (e.g., Cape Canaveral)
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# Coordinates for the closest coastline point
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate the distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create and add a folium.Marker on the selected closest coastline point
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_coastline),
    )
)

# Add the marker to the map
site_map.add_child(distance_marker)

# Display the map
site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [32]:
# Coordinates for a launch site (e.g., Cape Canaveral)
launch_site_coords = [28.573255, -80.646895]

# Coordinates for the closest coastline point
coastline_coords = [28.56367, -80.57163]

# Create a PolyLine object
lines = folium.PolyLine(locations=[launch_site_coords, coastline_coords], weight=1)

# Add the PolyLine to the map
site_map.add_child(lines)

# Display the map
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [33]:
from folium.plugins import MousePosition

# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add Mouse Position to the map
MousePosition(
    position='topright',
    separator=' | ',
    empty_string='Unavailable',
    lng_first=True,
    num_digits=5,
    prefix='Coordinates:'
).add_to(site_map)

# Display the map
site_map


In [34]:
from folium.features import DivIcon

# Example coordinates for a launch site (e.g., Cape Canaveral)
launch_site_coords = [28.573255, -80.646895]

# Coordinates for points of interest
city_coords = [28.5383, -81.3792]  # Example: Orlando
railway_coords = [28.56367, -80.57163]  # Example: Closest railway
highway_coords = [28.5721, -80.5853]  # Example: Closest highway

# Calculate distances
distance_city = calculate_distance(launch_site_coords[0], launch_site_coords[1], city_coords[0], city_coords[1])
distance_railway = calculate_distance(launch_site_coords[0], launch_site_coords[1], railway_coords[0], railway_coords[1])
distance_highway = calculate_distance(launch_site_coords[0], launch_site_coords[1], highway_coords[0], highway_coords[1])

# Create and add markers to the map
city_marker = folium.Marker(
    location=city_coords,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_city),
    )
)

railway_marker = folium.Marker(
    location=railway_coords,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_railway),
    )
)

highway_marker = folium.Marker(
    location=highway_coords,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_highway),
    )
)

# Add markers to the map
site_map.add_child(city_marker)
site_map.add_child(railway_marker)
site_map.add_child(highway_marker)


In [35]:
# Create PolyLine objects
city_line = folium.PolyLine(locations=[launch_site_coords, city_coords], weight=1)
railway_line = folium.PolyLine(locations=[launch_site_coords, railway_coords], weight=1)
highway_line = folium.PolyLine(locations=[launch_site_coords, highway_coords], weight=1)

# Add lines to the map
site_map.add_child(city_line)
site_map.add_child(railway_line)
site_map.add_child(highway_line)

# Display the map
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
